In [1]:
from igraph import *
import pickle
import folium
import shapely.geometry as sg
import pandas as pd
import numpy as np
from pyproj import Proj, Transformer
import pycrs

In [2]:
with (open("graph_clusters_30m_biasCorr_bos.pk", "rb")) as file:
    g=pickle.load(file, encoding='latin1')

In [3]:
points=sg.MultiPoint(g.vs["pos"])

cvhull=points.convex_hull

#cvhull.boundary.coords[:]

In [4]:
lats=[c[0] for c in g.vs["pos"]]
longs=[c[1] for c in g.vs["pos"]]

min_lat=min(lats)
min_long=min(longs)

max_lat=max(lats)
max_long=max(longs)

In [5]:
min_lat, min_long, max_lat, max_long

(42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569)

### Looking at map features

In [6]:
map_features_or=pd.read_csv("map_features_full_area_added_features.csv").drop_duplicates(subset='id')
map_features=pd.read_csv('map_features_full_area.csv')

In [7]:
map_features=pd.concat([map_features,map_features_or]).drop_duplicates('id').drop('Unnamed: 0',axis=1)

In [8]:
map_features

,id,key,value,latitude,longitude
0,7209589252,shop,supermarket,42.194485,-71.287765
1,6802507055,shop,hairdresser,42.223623,-71.223304
2,8274346923,shop,jewelry,42.192524,-71.201621
3,6802507057,shop,beauty,42.223790,-71.223172
4,7209693823,shop,clothes,42.193919,-71.288965
...,...,...,...,...,...
865,4012279665,amenity,fast_food,42.421293,-71.005446
866,4012279666,amenity,fast_food,42.421988,-71.005344
867,6322303590,amenity,fast_food,42.463934,-70.943908
868,6611561722,amenity,fast_food,42.464039,-70.943874


In [9]:
sorted(map_features['value'].unique().tolist())

['alcohol',
 'alternative',
 'anticlockwise',
 'antiques',
 'apartments',
 'appliance',
 'art',
 'arts_centre',
 'artwork',
 'atm',
 'attraction',
 'audiologist',
 'bag',
 'bakery',
 'bank',
 'bar',
 'bbq',
 'beauty',
 'bed',
 'bench',
 'beverages',
 'bicycle',
 'biergarten',
 'boat',
 'bookmaker',
 'books',
 'boutique',
 'butcher',
 'cafe',
 'candles',
 'cannabis',
 'car',
 'carpet',
 'charity',
 'chemist',
 'childcare',
 'chocolate',
 'cinema',
 'clinic',
 'clock',
 'clothes',
 'coffee',
 'college',
 'commercial',
 'computer',
 'confectionery',
 'conservation',
 'construction',
 'convenience',
 'copyshop',
 'cosmetics',
 'counselling',
 'courthouse',
 'craft',
 'crossing',
 'curtain',
 'deli',
 'dentist',
 'derail',
 'designated',
 'diplomatic',
 'dismount',
 'doctors',
 'doityourself',
 'e-cigarette',
 'electronics',
 'elevator',
 'erotic',
 'fabric',
 'fast_food',
 'firepit',
 'flooring',
 'florist',
 'fountain',
 'frame',
 'fuel',
 'furniture',
 'gas',
 'general',
 'gift',
 'gover

In [10]:
to_remove=["cutting","aeroway","fuel","oven","power","access","type","horse","military","oneway","route","highway","recycling type"]

#map_features=map_features[~map_features["key"].isin(to_remove)]

In [11]:
map_features.max()

id           8565749424
key              tunnel
value               yes
latitude      42.481999
longitude    -70.917046
dtype: object

In [12]:
map_features.min()

id            36303521
key             access
value          alcohol
latitude     42.189846
longitude   -71.291214
dtype: object

In [13]:
from folium import plugins

points = (map_features.latitude,map_features.longitude)
coordinates =[]
# Setting lat and long 
lat = points[0]
long = points[1]
# Latitude and longitude that will open map. Here I put central of city São Paulo
mapa = folium.Map(location=[0.5*min_lat+0.5*max_lat, 0.5*min_long+0.5*max_long])
# Append latitude and longitude coordinates array
for la,lo in zip(lat,long):
    coordinates.append([la,lo])
for coord in coordinates[::100]:
    
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapa )
folium.plugins.HeatMap(coordinates).add_to(mapa)

#### A visualization of the total area in question along with a heatmap of the some of the map features

In [14]:
#mapa

### Scaling inside the area

In [15]:
map_features=map_features[map_features["latitude"]>min_lat]
map_features=map_features[map_features["latitude"]<max_lat]
map_features=map_features[map_features["longitude"]>min_long]
map_features=map_features[map_features["longitude"]<max_long]

In [16]:
map_features

,id,key,value,latitude,longitude
0,7209589252,shop,supermarket,42.194485,-71.287765
1,6802507055,shop,hairdresser,42.223623,-71.223304
2,8274346923,shop,jewelry,42.192524,-71.201621
3,6802507057,shop,beauty,42.223790,-71.223172
4,7209693823,shop,clothes,42.193919,-71.288965
...,...,...,...,...,...
865,4012279665,amenity,fast_food,42.421293,-71.005446
866,4012279666,amenity,fast_food,42.421988,-71.005344
867,6322303590,amenity,fast_food,42.463934,-70.943908
868,6611561722,amenity,fast_food,42.464039,-70.943874


# 2) Calculating projected coordinates

In [17]:
def mass2001_wgs84(points): # transform coordinate systems for a list of points
    # points: [lat lon]
    points = np.array(points)

    # set the from Proj
    inProj = Proj('epsg:4326') # wgs84
    # read the to projection from the .prj file
    tocrs = pycrs.load.from_file(r'juncs_greaterBos_massProj.prj')
    tocrs_proj4 = tocrs.to_proj4()
    outProj = Proj(tocrs_proj4) # state plan massachusetts 2001 meter
    
    x1s = [a[0] for a in points]
    y1s = [a[1] for a in points]
    
    transformer = Transformer.from_proj('epsg:4326', outProj)
    x2s,y2s=transformer.transform(x1s,y1s)
    #x2s,y2s = transform(outProj,inProj,x1s,y1s)
    #print(x2s,y2s)
    points_proj = list(zip(y2s,x2s)) # be careful
    points_proj = [[a[0], a[1]] for a in list(zip(x2s,y2s))] # be careful
    return (points_proj)


In [18]:
##This one in case you don't have the projection file! It just has a veery small error (mm)


def project(latlon):
    #Local Azimuthal Equidistant projection
    lat_0,lon_0 = np.array(latlon).mean(axis=0)

    lat,lon = np.array(latlon).T

    crs_aeqd = pyproj.Proj(proj='aeqd', lon_0=lon_0, lat_0=lat_0, units='m') # Local Azimuthal Equidistant projection

    return np.column_stack((crs_aeqd.transform(lon,lat)))

In [19]:
#g.vs["ppos"]

In [20]:
#list(mass2001_wgs84(g.vs["pos"]))

In [21]:
#np.around(np.array(mass2001_wgs84(g.vs["pos"])), decimals=8)==np.array(g.vs["ppos"])

In [22]:
#with open('Boston_street_network_with_pyproj3.0.pickle', 'wb') as handle:
#    pickle.dump(g, handle)

The function works fine, you can check by uncommenting the cells above

#### Transforming map_features coordinates

In [23]:
projected=np.array(mass2001_wgs84(map_features[["latitude","longitude"]]))
map_features["proj_latitude"]=projected[:,0]
map_features["proj_longitude"]=projected[:,1]

In [24]:
map_features.head()

,id,key,value,latitude,longitude,proj_latitude,proj_longitude
0,7209589252,shop,supermarket,42.194485,-71.287765,217529.572236,882692.814751
1,6802507055,shop,hairdresser,42.223623,-71.223304,222843.187503,885944.558560
2,8274346923,shop,jewelry,42.192524,-71.201621,224645.314136,882496.331576
3,6802507057,shop,beauty,42.223790,-71.223172,222854.066064,885963.143264
4,7209693823,shop,clothes,42.193919,-71.288965,217430.589217,882629.701576


# 3) Matching

In [25]:
ed = [e.tuple for e in g.es]

edp = np.array([(g.vs[i]["ppos"],g.vs[j]["ppos"]) for i,j in ed])

# Creating a list of LineString
linestring_list=[]
for i in range(len(edp)):
    linestring_list.append(sg.LineString([edp[i][0], edp[i][1]]))

### Matching function

In [26]:
def match_to_edge(x, y):
    #filtering to the nearest km
    d0 = 1000**2
    p0 = np.array([x,y])[np.newaxis]
    possible_edges=np.where((((edp-p0)**2).sum(axis=2)<d0).all(axis=1))[0]

    #Finding smallest distance
    point=sg.Point(x, y)
    linestring_list_subset=[linestring_list[i] for i in possible_edges]
    distances=[point.distance(linestring) for linestring in linestring_list_subset]
    if len(distances)!=0:
        return possible_edges[distances.index(min(distances))]
    else:
        return None


In [27]:
#map_features[["proj_latitude","proj_longitude"]].apply(lambda x: match_to_edge(x["proj_latitude"], x["proj_longitude"]), axis=1)

In [28]:
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=3)

import time
a=time.time()

map_features["edge"]=map_features[["proj_latitude","proj_longitude"]].parallel_apply(lambda x: match_to_edge(x["proj_latitude"], x["proj_longitude"]), axis=1)

b=time.time()
print(b-a)

INFO: Pandarallel will run on 3 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
191.79673409461975


In [29]:
map_features[map_features["edge"].isna()]

,id,key,value,latitude,longitude,proj_latitude,proj_longitude,edge
26556,7616117755,amenity,school,42.317903,-71.008653,240503.641575,896496.224436,NaN


In [244]:
# map_features.to_csv('map_feats_bel_edge.csv',index=False)

In [30]:
map_features.dropna(inplace=True)
map_features["edge"]=map_features["edge"].astype(int)

In [51]:
# lim=[42.359348459262,-71.0669727000319,42.36109346267096,-71.0659250121868]

Let's see if it managed to match the nodes to the edges

In [74]:
latlon=[]
points=[]
for i in range(len(map_features[150::1000])):
    start=g.vs[g.es[map_features["edge"].iloc[i]].source]["pos"]
    end=g.vs[g.es[map_features["edge"].iloc[i]].target]["pos"]
    latlon.append((start, end))
    points.append((map_features.iloc[i]["latitude"], map_features.iloc[i]["longitude"]))
    

mapit = folium.Map( location=[start[0], start[1]], zoom_start=10 )
#latlon = [start, end]
#folium.TileLayer('cartodbpositron').add_to(mapit)
#folium.TileLayer('cartodbdark_matter').add_to(mapit)

for i in range(len(latlon)):
    street=latlon[i]
    colors=["black","blue","green","black"]
    folium.Marker( location=[ street[0][0], street[0][1] ], icon = folium.Icon(color=colors[0],icon='jpy'), radius=8 ).add_to( mapit )
    folium.Marker( location=[ street[1][0], street[1][1] ], icon = folium.Icon(color=colors[0],icon='menu_left'), radius=8 ).add_to( mapit )
    folium.CircleMarker( location=[ points[i][0],  points[i][1] ], icon = folium.Icon(color=colors[1]), radius=8 ).add_to( mapit )

In [93]:
# p=np.array(g.vs['pos'])

In [245]:
# g.es[7364].source, g.es[7364].target

In [246]:
# [e for e in g.es.select(_source=3638, _target=4093)]

In [247]:
# np.where((p[:,0]>42.3504)*(p[:,0]<42.3514)*(p[:,1]>-71.0753)*(p[:,1]<-71.0728)==True)

In [249]:
# map_features_=map_features[map_features['edge']==68234]
# map_features2_=map_features[map_features['edge']==109926]

In [241]:
latlon=[]
points=[]
for i in range(len(map_features_)):
    start=g.vs[g.es[map_features_["edge"].iloc[i]].source]["pos"]
    end=g.vs[g.es[map_features_["edge"].iloc[i]].target]["pos"]
    latlon.append((start, end))
    points.append((map_features_.iloc[i]["latitude"], map_features_.iloc[i]["longitude"]))
    
latlon2=[]
points2=[]
for i in range(len(map_features2_)):
    start=g.vs[g.es[map_features2_["edge"].iloc[i]].source]["pos"]
    end=g.vs[g.es[map_features2_["edge"].iloc[i]].target]["pos"]
    latlon2.append((start, end))
    points2.append((map_features2_.iloc[i]["latitude"], map_features2_.iloc[i]["longitude"]))
    
    
    
mapit = folium.Map( location=[start[0], start[1]], zoom_start=18 )
#latlon = [start, end]
#folium.TileLayer('cartodbpositron').add_to(mapit)
#folium.TileLayer('cartodbdark_matter').add_to(mapit)

for i in range(len(latlon)):
    street=latlon[i]
    colors=["black","white","green","black"]
    folium.Marker( location=[ street[0][0], street[0][1] ], icon = folium.Icon(color=colors[0],icon='jpy'), radius=8 ).add_to( mapit )
    folium.Marker( location=[ street[1][0], street[1][1] ], icon = folium.Icon(color=colors[0],icon='menu_left'), radius=8 ).add_to( mapit )
    folium.CircleMarker(color = 'black', location=[ points[i][0],  points[i][1] ], icon = folium.Icon(color='red'), radius=8 ).add_to( mapit )
    
    
for i in range(len(latlon2)):
    street=latlon2[i]
    colors=["red","blue","green","black"]
    folium.Marker( location=[ street[0][0], street[0][1] ], icon = folium.Icon(color=colors[0],icon='jpy'), radius=8 ).add_to( mapit )
    folium.Marker( location=[ street[1][0], street[1][1] ], icon = folium.Icon(color=colors[0],icon='menu_left'), radius=8 ).add_to( mapit )
    folium.CircleMarker(color='red', location=[ points2[i][0],  points2[i][1] ], icon = folium.Icon(color=colors[1]), radius=8 ).add_to( mapit )

In [242]:
mapit

In [243]:
# mapit.save("matched_boylston_newbury.html")

In [237]:
if 'Unnamed: 0' in map_features.columns.tolist():
    map_features.drop('Unnamed: 0',axis=1,inplace=True)

In [238]:
map_features

,id,key,value,latitude,longitude,proj_latitude,proj_longitude,edge
0,7209589252,shop,supermarket,42.194485,-71.287765,217529.572236,882692.814751,92122
1,6802507055,shop,hairdresser,42.223623,-71.223304,222843.187503,885944.558560,10015
2,8274346923,shop,jewelry,42.192524,-71.201621,224645.314136,882496.331576,91523
3,6802507057,shop,beauty,42.223790,-71.223172,222854.066064,885963.143264,10015
4,7209693823,shop,clothes,42.193919,-71.288965,217430.589217,882629.701576,47276
...,...,...,...,...,...,...,...,...
865,4012279665,amenity,fast_food,42.421293,-71.005446,240701.401394,907981.801605,71068
866,4012279666,amenity,fast_food,42.421988,-71.005344,240709.355968,908059.115140,90526
867,6322303590,amenity,fast_food,42.463934,-70.943908,245734.985170,912749.453902,22434
868,6611561722,amenity,fast_food,42.464039,-70.943874,245737.697072,912761.146290,91709


Points matched!

# 4) Adding as attributes

In [239]:
group_by_key=pd.DataFrame(map_features.groupby(["edge","key"]).agg('count')["id"])
group_by_value=pd.DataFrame(map_features.groupby(["edge","value"]).agg('count')["id"])

In [240]:
group_by_key.head()

,,id
edge,key,
13,highway,2
36,highway,1
38,amenity,1
39,highway,1
41,access,2


In [241]:
for edge_id in group_by_key.index.get_level_values('edge').unique():
    for key in group_by_key.index.get_level_values('key').unique():
        if (edge_id, key) in group_by_key.index:
            g.es[edge_id][key]=group_by_key.loc[(edge_id,key)]["id"]
            
    for value in group_by_value.index.get_level_values('value').unique():
        if (edge_id, value) in group_by_value.index:
            g.es[edge_id][value]=group_by_value.loc[(edge_id,value)]["id"]


In [242]:
g.es[71726]

igraph.Edge(<igraph.Graph object at 0x7f9b427e49a0>, 71726, {'ppos': array([236011.13743065, 900173.88519221]), 'weight': 61.562, 'highway': 3, 'crossing': 3, 'amenity': 8, 'parking': None, 'access': None, 'no': None, 'shop': None, 'deli': None, 'restaurant': 7, 'stop': None, 'fuel': None, 'cafe': None, 'clothes': None, 'hairdresser': None, 'oneway': None, 'railway': None, 'switch': None, 'yes': None, 'library': None, 'bench': None, 'books': None, 'pub': None, 'convenience': None, 'laundry': None, 'dentist': None, 'private': None, 'school': None, 'bicycle': None, 'confectionery': None, 'tourism': None, 'hotel': None, 'bbq': None, 'bar': None, 'bank': None, 'furniture': None, 'place': None, 'hamlet': None, 'bakery': None, 'shelter': None, 'mall': None, 'emergency': None, 'phone': None, 'supermarket': None, 'childcare': None, 'theatre': None, 'sports': None, 'healthcare': None, 'alternative': None, 'beauty': None, 'optician': None, 'alcohol': None, 'pet': None, 'square': None, 'recycling

In [243]:
with open('Boston_network_matched.pickle', 'wb') as handle:
    pickle.dump(g, handle)

In [250]:
max([l[i] for i in range(len(l)) if l[i]!=None])

8

In [248]:
'mall' in map_features['value'].tolist()

True

In [249]:
l=g.es['fast_food']

# Dealing with parks and green

In [4]:
import pickle as pk
import scipy.stats as st
import matplotlib.pyplot
import igraph as ig
import numpy as np
import pandas as pd
from collections import Counter
import ast

# import PedProj

import shapely.geometry as sg

with open('SF_network_matched.pickle','rb') as handle:
    g = pk.load(handle)


In [5]:
g.es.attributes()

['weight',
 'amenity',
 'highway',
 'railway',
 'shop',
 'crossing',
 'dentist',
 'hardware',
 'optician',
 'stop',
 'bench',
 'bicycle',
 'no',
 'switch',
 'beauty',
 'cafe',
 'laundry',
 'platform',
 'restaurant',
 'tourism',
 'alcohol',
 'art',
 'attraction',
 'clothes',
 'fast_food',
 'gift',
 'hairdresser',
 'pub',
 'shoes',
 'tattoo',
 'tobacco',
 'watches',
 'clinic',
 'cosmetics',
 'supermarket',
 'wine',
 'books',
 'frame',
 'doityourself',
 'boutique',
 'convenience',
 'telephone',
 'parking',
 'farm',
 'toilets',
 'access',
 'private',
 'bar',
 'artwork',
 'bank',
 'atm',
 'fountain',
 'chocolate',
 'place',
 'hamlet',
 'hotel',
 'furniture',
 'school',
 'station',
 'kindergarten',
 'butcher',
 'tyres',
 'pharmacy',
 'police',
 'doctors',
 'games',
 'fuel',
 'greengrocer',
 'sports',
 'chemist',
 'lighting',
 'elevator',
 'library',
 'veterinary',
 'theatre',
 'building',
 'construction',
 'shelter',
 'office',
 'government',
 'massage',
 'outdoor',
 'stationery',
 'electron

In [ ]:

ppos = np.array(g.vs["ppos"])
pos = np.array(g.vs["pos"])    
#links
ed = [e.tuple for e in g.es]

#center of the links
edc = np.array([(ppos[i]+ppos[j])/2 for i,j in ed])

#Linestring edges
edp = [sg.LineString([ppos[i],ppos[j]]) for i,j in ed]


P = pd.read_csv('parks_SF.csv')
P = P.drop_duplicates()

# select one polygon
s = P['3'].iloc[0]

Px = np.array(ast.literal_eval(s))[:,1:]

poly = sg.Polygon(PedProj.wgs84_mass2001(Px))


buffer = 20.
##### Find Diameter of the polygon to search the edges
box = poly.minimum_rotated_rectangle
# get coordinates of polygon vertices
x, y = box.exterior.coords.xy

# get length of bounding box edges
edge_length = (sg.Point(x[0], y[0]).distance(sg.Point(x[1], y[1])),
               sg.Point(x[1], y[1]).distance(sg.Point(x[2], y[2])))

# get length of polygon as the longest edge of the bounding box
length = max(edge_length)


#### Center of the polygon
p0 = np.array(poly.centroid.coords[:])


#Find possible overlapping links
idx = np.where(np.linalg.norm(edc-p0,axis=1)<length)[0]

#buffered Poligon
polyb = poly.buffer(buffer)


#Overlap edges
sorted([(edp[i].intersection(polyb).length/edp[i].length,ed[i]) for i in idx],reverse=True)